In [ ]:
!pip install gradio
!python -m pip install git+https://github.com/huggingface/transformers

  Cloning https://github.com/huggingface/transformers to /tmp/pip-req-build-mlsc9w8w
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers /tmp/pip-req-build-mlsc9w8w
  Resolved https://github.com/huggingface/transformers to commit f38590dade57c1f8cf8a67e9409dae8935f8c478
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
# Original

from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from PIL import Image
import requests

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct"
)

messages = [
        {
            "role": "system",
            "content" : [
                {
                    "type": "image",
                },
                {
                    "type": "text",
                    "text": "You are an conversation image recognition chatbot. Communicate with humans using natural language. Recognize the images, have a spatial understanding and answer the questions in a concise manner. Generate the best response for a user query. It must be correct lexically and grammatically.",
                }
            ]
        }
    ]

def generate_output_text(image, user_input):

    messages.append({
        "role": "user",
        "content" : [
            {
                "type": "text",
                "text": user_input
            }
        ]
    })

    text_prompt = processor.apply_chat_template(messages, add_generation_prompt=True)

    inputs = processor(
        text = [text_prompt],
        images = [image],
        padding = True,
        return_tensors = "pt"
    )

    inputs = inputs.to("cuda")

    output_ids = model.generate(**inputs, max_new_tokens=1024)

    generated_ids = [
        output_ids[len(input_ids) :]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]

    bot_output = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    messages.append({
        "role": "assistant",
        "content" : [
            {
                "type": "text",
                "text": bot_output
            }
        ]
    })

    return bot_output

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/1.20k [00:00<?, ?B/s]

Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}


model.safetensors.index.json:   0%|          | 0.00/56.4k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.99G [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/2.78M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/1.67M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/7.03M [00:00<?, ?B/s]

chat_template.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

In [ ]:
# try 1

from transformers import Qwen2VLForConditionalGeneration, AutoTokenizer, AutoProcessor
from PIL import Image
import requests

model = Qwen2VLForConditionalGeneration.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct",
    torch_dtype="auto",
    device_map="auto",
)

processor = AutoProcessor.from_pretrained(
    "Qwen/Qwen2-VL-2B-Instruct"
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Unrecognized keys in `rope_scaling` for 'rope_type'='default': {'mrope_section'}
`Qwen2VLRotaryEmbedding` can now be fully parameterized by passing the model config through the `config` argument. All other arguments will be removed in v4.46


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
import gradio as gr
import random
import time

with gr.Blocks() as demo:

    with gr.Row():

        with gr.Column(scale=1):

            gr.Markdown("## Upload an Image")
            image_input = gr.Image(type="pil", label="Upload your image here")

        with gr.Column(scale=2):

            chatbot = gr.Chatbot()
            msg = gr.Textbox()
            clear = gr.ClearButton([msg, chatbot])

            def user(message, chat_history):
                return "", chat_history + [[message, ""]]

            global history
            history = [
                        {
                            "role": "system",
                            "content" : [
                                {
                                    "type": "image",
                                },
                                {
                                    "type": "text",
                                    "text": "You are an conversation image recognition chatbot. Communicate with humans using natural language. Recognize the images, have a spatial understanding and answer the questions in a concise manner. Generate the best response for a user query. It must be correct lexically and grammatically.",
                                }
                            ]
                        }
                    ]

            def init_history():
                history = [
                        {
                            "role": "system",
                            "content" : [
                                {
                                    "type": "image",
                                },
                                {
                                    "type": "text",
                                    "text": "You are an conversation image recognition chatbot. Communicate with humans using natural language. Recognize the images, have a spatial understanding and answer the questions in a concise manner. Generate the best response for a user query. It must be correct lexically and grammatically.",
                                }
                            ]
                        }
                    ]
                return None

            def chat_function(image, chat_history):

                message = ''

                if chat_history[-1][0] is not None:
                    message = str(chat_history[-1][0])

                history.append({
                    "role": "user",
                    "content" : [
                        {
                            "type": "text",
                            "text": message
                        }
                    ]
                })

                text_prompt = processor.apply_chat_template(history, add_generation_prompt=True)

                inputs = processor(
                    text = [text_prompt],
                    images = [image],
                    padding = True,
                    return_tensors = "pt"
                )

                inputs = inputs.to("cuda")

                output_ids = model.generate(**inputs, max_new_tokens=1024)

                generated_ids = [
                    output_ids[len(input_ids) :]
                    for input_ids, output_ids in zip(inputs.input_ids, output_ids)
                ]

                bot_output = processor.batch_decode(
                    generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
                )

                history.append({
                    "role": "assistant",
                    "content" : [
                        {
                            "type": "text",
                            "text": bot_output
                        }
                    ]
                })

                bot_output_str = str(bot_output).replace('"', '').replace('[', '').replace(']', '')
                for character in bot_output_str:
                    chat_history[-1][1] += character
                    time.sleep(0.05)
                    yield chat_history

            msg.submit(user, [msg, chatbot], [msg, chatbot], queue=False).then(chat_function, [image_input, chatbot], [chatbot])
            clear.click(init_history, None, chatbot, queue=False)

In [ ]:
demo.launch(debug=True)

Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://75ac6b29305c380482.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


In [ ]:
history

In [ ]:
import gradio as gr
from PIL import Image

history = [
                {
                    "role": "system",
                    "content" : [
                        {
                            "type": "image",
                        },
                        {
                            "type": "text",
                            "text": "You are an conversation image recognition chatbot. Communicate with humans using natural language. Recognize the images, have a spatial understanding and answer the questions in a concise manner. Generate the best response for a user query. It must be correct lexically and grammatically.",
                        }
                    ]
                }
            ]

def chat_function(message, history_dumb, image_input_cbt):
    if image_input_cbt is None:
        return "Please upload an image before starting the chat."

    # Append user message to history
    history.append({
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": message
            }
        ]
    })

    text_prompt = processor.apply_chat_template(history, add_generation_prompt=True)

    # Convert the image to tensors and process it
    inputs = processor(
        text=[text_prompt],
        images=[image_input_cbt],  # Use the image passed by Gradio
        padding=True,
        return_tensors="pt"
    )

    inputs = inputs.to("cuda")

    output_ids = model.generate(**inputs, max_new_tokens=1024)

    generated_ids = [
        output_ids[len(input_ids):]
        for input_ids, output_ids in zip(inputs.input_ids, output_ids)
    ]

    bot_output = processor.batch_decode(
        generated_ids, skip_special_tokens=True, clean_up_tokenization_spaces=True
    )

    # Append the bot response to the history
    history.append({
        "role": "assistant",
        "content": [
            {
                "type": "text",
                "text": bot_output
            }
        ]
    })

    return bot_output

# Define the Gradio interface
with gr.Blocks() as demo:
    with gr.Row():
        with gr.Column():
            image_input = gr.Image(type="pil", label="Upload your image here")
        with gr.Column():
            message_input = gr.Textbox(label="Enter your message here")
            chatbot_output = gr.Textbox(label="Chatbot Output")
            submit_button = gr.Button("Submit")

    def handle_chat(image_input_cbt, message_input_cbt):
        history = []
        return chat_function(message_input_cbt, history, image_input_cbt)

    # Define the interaction
    submit_button.click(fn=handle_chat, inputs=[image_input, message_input], outputs=chatbot_output)

# Launch the Gradio app
demo.launch()
